In [6]:
import pandas as pd
import numpy as np

In [ ]:
tennis_df = pd.read_csv("data/atp_matches_1990.csv")

In [8]:
year = 1991

In [ ]:
while year < 2025:
    temp = pd.read_csv(f"data/atp_matches_{year}.csv")
    tennis_df = pd.concat([tennis_df, temp], ignore_index=True)
    year += 1

In [ ]:
new_columns = {col: col.replace('winner', 'w') for col in tennis_df.columns if col.startswith('winner')}


In [ ]:
new_columns.update({col: col.replace('loser', 'l') for col in tennis_df.columns if col.startswith("loser")})

In [ ]:
tennis_df = tennis_df.rename(columns=new_columns)

In [ ]:
winner_cols = [col for col in tennis_df.columns if col.startswith('w')]
loser_cols = [col for col in tennis_df.columns if col.startswith('l')]
other_cols = [col for col in tennis_df.columns if col not in winner_cols and col not in loser_cols]

In [14]:
stat_names = [col.replace('w_', '') for col in winner_cols]

In [ ]:
swap_mask = np.random.rand(len(tennis_df)) < 0.5

In [16]:
new_df = pd.DataFrame()

In [ ]:
for stat in stat_names:
    winner_stat_col = f'w_{stat}'
    loser_stat_col = f'l_{stat}'
    
    new_df[f'p1_{stat}'] = np.where(swap_mask, tennis_df[loser_stat_col], tennis_df[winner_stat_col])
    
    new_df[f'p2_{stat}'] = np.where(swap_mask, tennis_df[winner_stat_col], tennis_df[loser_stat_col])

In [18]:
new_df['winner'] = np.where(swap_mask, 'p2', 'p1')

In [ ]:
for col in other_cols:
    new_df[col] = tennis_df[col]

In [20]:
new_df['tourney_date'] = pd.to_datetime(new_df['tourney_date'], format='%Y%m%d')

In [21]:
players = pd.read_csv("data/atp_players.csv")

/var/folders/3m/83d_phts1yg9dsv29rxnq4n80000gn/T/ipykernel_72649/816518900.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("data/atp_players.csv")


In [22]:
new_df = new_df.sort_values(by='tourney_date').reset_index(drop=True)

In [23]:
p_ids = new_df[['p1_id', 'p2_id']].astype(str)
new_df['matchup_id'] = p_ids.min(axis=1) + '-' + p_ids.max(axis=1)

In [24]:
new_df['low_won'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] < new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] < new_df['p1_id']))).astype(int)
new_df['high_won'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] > new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] > new_df['p1_id']))).astype(int)

In [25]:
cumulative_wins = new_df.groupby('matchup_id')[['low_won', 'high_won']].cumsum()

In [26]:
new_df['high_won'] = cumulative_wins['high_won']
new_df['low_won'] =  cumulative_wins['low_won']

In [27]:
new_df['low_won'] = new_df.groupby('matchup_id')['low_won'].shift(1).fillna(0).astype(int)
new_df['high_won'] = new_df.groupby('matchup_id')['high_won'].shift(1).fillna(0).astype(int)

In [28]:
#djokovic 104925 nadal 104745

In [29]:
new_df = new_df.sort_values(by="tourney_date").reset_index(drop=True)

In [30]:
new_df['low_won_surface'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] < new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] < new_df['p1_id']))).astype(int)
new_df['high_won_surface'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] > new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] > new_df['p1_id']))).astype(int)

In [31]:
cumulative_surface_wins = new_df.groupby(['matchup_id', "surface"])[['low_won_surface', 'high_won_surface']].cumsum()

In [32]:
new_df['high_won_surface'] = cumulative_surface_wins['high_won_surface']
new_df['low_won_surface'] =  cumulative_surface_wins['low_won_surface']

In [33]:
new_df['low_won_surface'] = new_df.groupby(['matchup_id', 'surface'])['low_won_surface'].shift(1).fillna(0).astype(int)
new_df['high_won_surface'] = new_df.groupby(['matchup_id', 'surface'])['high_won_surface'].shift(1).fillna(0).astype(int)

In [34]:
# TODO write the ELO methods, one for win differential, one for k-score, one for update and one to initialize

In [77]:
import re

def calculate_winner_game_differential(score_string: str) -> int:
    total_differential = 0
    sets = score_string.split(' ')

    for set_score in sets:
        clean_set_score = re.sub(r'\(.*\)', '', set_score).strip()

        if clean_set_score.upper() == "RET":
            return 0

        try:
            winner_games_str, loser_games_str = clean_set_score.split('-')
            winner_games = int(winner_games_str)
            loser_games = int(loser_games_str)

            total_differential += (winner_games - loser_games)
        except ValueError:
            print(f"Warning: Could not parse set score '{set_score}'. Skipping this set.")
            continue
        except IndexError:
            print(f"Warning: Malformed set score '{set_score}'. Skipping this set.")
            continue

    return total_differential

In [87]:
players['elo'] = 1500
players['matches_played'] = 0
players['elo_clay'] = 1500
players['elo_hard'] = 1500
players['elo_grass'] = 1500
players['elo_carpet'] = 1500
players['svpt'] = 0
players['1st_in'] = 0
players['1st_won'] = 0
players['2nd_won'] = 0
players['ace'] = 0
players['df'] = 0
players['bp_sv'] = 0
players['bp_won'] = 0
players['bp_faced'] = 0


In [95]:
new_df['p1_elo'] = 1500
new_df['p1_elo_clay'] = 1500
new_df['p1_elo_hard'] = 1500
new_df['p1_elo_grass'] = 1500
new_df['p1_elo_carpet'] = 1500
new_df['p1_1st_in_pct'] = 0
new_df['p1_1st_won_pct'] = 0
new_df['p1_2nd_won_pct'] = 0
new_df['p1_ace_pct'] = 0
new_df['p1_df_pct'] = 0
new_df['p1_bp_sv_pct'] = 0
new_df['p1_bp_won_pct'] = 0

new_df['p2_elo'] = 1500
new_df['p2_elo_clay'] = 1500
new_df['p2_elo_hard'] = 1500
new_df['p2_elo_grass'] = 1500
new_df['p2_elo_carpet'] = 1500
new_df['p2_1st_in_pct'] = 0
new_df['p2_1st_won_pct'] = 0
new_df['p2_2nd_won_pct'] = 0
new_df['p2_ace_pct'] = 0
new_df['p2_df_pct'] = 0
new_df['p2_bp_sv_pct'] = 0
new_df['p2_bp_won_pct'] = 0

In [66]:
players.loc[players['player_id'] == 104745]['name_first'].iloc[0] == "rafael"

False

In [80]:
def calculate_win_multiplier(score_string, winner_elo, loser_elo):
    denominator = 2.9
    game_differential = calculate_winner_game_differential(score_string)
    return np.log((game_differential / denominator) + 1) * (2.2 / ((winner_elo - loser_elo) * 0.001 + 2.2))

In [81]:
def calculate_k_factor(player_id, win_mult):
    matches_played = players.loc[players['player_id'] == int(player_id)]['matches_played'].iloc[0]
    if win_mult is None:
        return 800/ matches_played
    return win_mult * (800/matches_played)

In [82]:
def update_elos(p1_id, p2_id, winner, score_string):
    p1_elo = players.loc[players['p1_id'] == int(p1_id)]['elo'].iloc[0]
    p2_elo = players.loc[players['p2_id'] == int(p1_id)]['elo'].iloc[0]
    winner_elo, loser_elo = p1_elo, p2_elo if winner == "p1" else p2_elo, p1_elo

    win_multiplier = calculate_win_multiplier(score_string, winner_elo, loser_elo)

    p1_k_factor = calculate_k_factor(p1_id, win_multiplier)
    p2_k_factor = calculate_k_factor(p2_id, win_multiplier)

    p1_prob = 1 / (1 + (10 ** ((p2_elo - p1_elo) / 400)))
    p2_prob = 1 - p1_prob

    p1_score, p2_score = 1,0 if winner == "p1" else 0,1

    p1_new_elo = p1_elo + p1_k_factor * (p1_score - p1_prob)
    p2_new_elo = p2_elo + p2_k_factor * (p2_score - p2_prob)

    return p1_new_elo, p2_new_elo

In [88]:
new_df.columns

Index(['p1_id', 'p2_id', 'p1_seed', 'p2_seed', 'p1_entry', 'p2_entry',
       'p1_name', 'p2_name', 'p1_hand', 'p2_hand', 'p1_ht', 'p2_ht', 'p1_ioc',
       'p2_ioc', 'p1_age', 'p2_age', 'p1_ace', 'p2_ace', 'p1_df', 'p2_df',
       'p1_svpt', 'p2_svpt', 'p1_1stIn', 'p2_1stIn', 'p1_1stWon', 'p2_1stWon',
       'p1_2ndWon', 'p2_2ndWon', 'p1_SvGms', 'p2_SvGms', 'p1_bpSaved',
       'p2_bpSaved', 'p1_bpFaced', 'p2_bpFaced', 'p1_rank', 'p2_rank',
       'p1_rank_points', 'p2_rank_points', 'winner', 'tourney_id',
       'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'match_num', 'score', 'best_of', 'round', 'minutes', 'matchup_id',
       'low_won', 'high_won', 'low_won_surface', 'high_won_surface'],
      dtype='object')

In [ ]:
def run_stats(df):
    for i, row in df.iterrows():
        p1_new_elo, p2_new_elo = update_elos(row['p1_id'], row['p2_id'], row['winner'], row['score'])
        df['p1_elo'] = players.loc[players['p1_id'] == int(row['p1_id'])]['elo'].iloc[0]
        df['p2_elo'] = players.loc[players['p2_id'] == int(row['p2_id'])]['elo'].iloc[0]

        df['p1_elo_clay'] = players.loc[players['p1_id'] == int(row['p1_id'])]['elo_clay'].iloc[0]
        df['p2_elo_clay'] = players.loc[players['p2_id'] == int(row['p2_id'])]['elo_clay'].iloc[0]

        df['p1_elo_grass'] = players.loc[players['p1_id'] == int(row['p1_id'])]['elo_grass'].iloc[0]
        df['p2_elo_grass'] = players.loc[players['p2_id'] == int(row['p2_id'])]['elo_grass'].iloc[0]

        df['p1_elo_hard'] = players.loc[players['p1_id'] == int(row['p1_id'])]['elo_hard'].iloc[0]
        df['p2_elo_hard'] = players.loc[players['p2_id'] == int(row['p2_id'])]['elo_hard'].iloc[0]

        df['p1_elo_carpet'] = players.loc[players['p1_id'] == int(row['p1_id'])]['elo_carpet'].iloc[0]
        df['p2_elo_carpet'] = players.loc[players['p2_id'] == int(row['p2_id'])]['elo_carpet'].iloc[0]


    ####### TODO Change the p*_elo_*sruface* to one column that computes the elo for the given surface 

    ### TODO Figure out computations for the pct stats and upadte in this function as well

        
        ['elo'].iloc[0]
        if i > 10:
            break

In [92]:
new_df.groupby(["matchup_id", 'surface']).get_group(('104745-104925', 'Clay')).iloc[0]

p1_id          104925
p2_id          104745
p1_seed           NaN
p2_seed           2.0
p1_entry          NaN
                ...  
2nd_won_pct         0
ace_pct             0
df_pct              0
bp_sv_pct           0
bp_won_pct          0
Name: 58450, Length: 67, dtype: object